# SQL exercises
---
Code and exercises partly based on:
- https://towardsdatascience.com/jupyter-magics-with-sql-921370099589
- https://github.com/catherinedevlin/ipython-sql
- https://www.youtube.com/watch?v=i35OSGXt7wk
- Beaulieu, A. (2009): *Einführung in SQL (2. Auflage).* Köln: O'Reilly Verlag GmbH & Co. KG

## 1) Connect to a database

In [2]:
# connect to a local sqlite database using sqlalchemy

from sqlalchemy.engine import create_engine
engine = create_engine('sqlite:///data/chinook.db')

In [3]:
# show all table names in database
print(engine.table_names())

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'sqlite_sequence', 'sqlite_stat1', 'tracks']


In [4]:
# show metadata of tables
from sqlalchemy import MetaData, Table

metadata = MetaData()
albums = Table('albums', metadata, autoload=True, autoload_with=engine)
print(repr(albums))

Table('albums', MetaData(bind=None), Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False), Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False), Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False), schema=None)


In [5]:
# Optional: read sql table into pandas dataframe
import pandas as pd

df = pd.read_sql('select * from albums', engine)
df.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


## 2) Use SQL commands within Jupyter Notebook

In [ ]:
#https://github.com/catherinedevlin/ipython-sql
#--> introduces a linewise %sql or cellwise %%sql magic

In [6]:
%load_ext sql

In [7]:
%%sql sqlite:///data/chinook.db
select * from albums LIMIT 5

Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [8]:
%sql sqlite:///data/chinook.db

'Connected: @data/chinook.db'

In [17]:
# %env DATABASE_URL=sqlite:///data/chinook.db

In [9]:
%sql SELECT * FROM albums LIMIT 5

 * sqlite:///data/chinook.db
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


## 3) Create new database

In [10]:
%sql sqlite:///data/my_new_db.db3

'Connected: @data/my_new_db.db3'

## 4) use: SELECT, WHERE, LIMIT

In [19]:
%%sql 
SELECT * from albums 
WHERE ArtistID=11 OR ArtistID=12

 * sqlite:///data/chinook.db
Done.


AlbumId,Title,ArtistId
14,Alcohol Fueled Brewtality Live! [Disc 1],11
15,Alcohol Fueled Brewtality Live! [Disc 2],11
16,Black Sabbath,12
17,Black Sabbath Vol. 4 (Remaster),12


In [39]:
%%sql 
SELECT * FROM albums 
WHERE ArtistID>=10 AND ArtistID<20

 * sqlite:///data/chinook.db
Done.


AlbumId,Title,ArtistId
13,The Best Of Billy Cobham,10
14,Alcohol Fueled Brewtality Live! [Disc 1],11
15,Alcohol Fueled Brewtality Live! [Disc 2],11
16,Black Sabbath,12
17,Black Sabbath Vol. 4 (Remaster),12
18,Body Count,13
19,Chemical Wedding,14
20,The Best Of Buddy Guy - The Millenium Collection,15
21,Prenda Minha,16
22,Sozinho Remix Ao Vivo,16


In [40]:
%%sql 
SELECT * FROM albums 
WHERE Length(Title)<5

 * sqlite:///data/chinook.db
Done.


AlbumId,Title,ArtistId
128,Coda,22
131,IV,22
151,Load,50
181,Ten,118
182,Vs.,118
206,Core,134
236,Pop,150
239,War,150


## 5) use: JOIN ON

In [44]:
# kartesisches Produkt (ohne ON)

%sql SELECT * FROM albums JOIN artists LIMIT 10

 * sqlite:///data/chinook.db
Done.


AlbumId,Title,ArtistId,ArtistId_1,Name
1,For Those About To Rock We Salute You,1,1,AC/DC
1,For Those About To Rock We Salute You,1,2,Accept
1,For Those About To Rock We Salute You,1,3,Aerosmith
1,For Those About To Rock We Salute You,1,4,Alanis Morissette
1,For Those About To Rock We Salute You,1,5,Alice In Chains
1,For Those About To Rock We Salute You,1,6,Antônio Carlos Jobim
1,For Those About To Rock We Salute You,1,7,Apocalyptica
1,For Those About To Rock We Salute You,1,8,Audioslave
1,For Those About To Rock We Salute You,1,9,BackBeat
1,For Those About To Rock We Salute You,1,10,Billy Cobham


In [45]:
%%sql 
SELECT AlbumId, Title, Name 
FROM albums JOIN artists 
ON albums.ArtistID = artists.ArtistID
LIMIT 10

 * sqlite:///data/chinook.db
Done.


AlbumId,Title,Name
1,For Those About To Rock We Salute You,AC/DC
2,Balls to the Wall,Accept
3,Restless and Wild,Accept
4,Let There Be Rock,AC/DC
5,Big Ones,Aerosmith
6,Jagged Little Pill,Alanis Morissette
7,Facelift,Alice In Chains
8,Warner 25 Anos,Antônio Carlos Jobim
9,Plays Metallica By Four Cellos,Apocalyptica
10,Audioslave,Audioslave
